In [2]:
#載入csv的檔案
import numpy as np
import pandas as pd
import pyprind
import pandas as pd
import os
import sys
from packaging import version
import urllib.request


df = pd.read_csv('movie_data.csv', encoding='utf-8')

# the following is necessary on some computers:
df = df.rename(columns={"0": "review", "1": "sentiment"})

df.head(3)

ModuleNotFoundError: No module named 'pandas'

In [ ]:
#使用CountVectorizer將文字轉換成一個詞頻矩陣，表示該詞彙出現的頻率
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
#建立CountVectorizer方法
count = CountVectorizer()
#建立矩陣
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
#bag回傳矩陣
bag = count.fit_transform(docs)

In [ ]:
print(count.vocabulary_)
#印出索引值

In [3]:
print(bag.toarray())
#印出矩陣內容

NameError: name 'bag' is not defined

In [ ]:
np.set_printoptions(precision=2)
#將浮點數的顯示精度設置為小數點後兩位

In [ ]:
#計算tfidf向量
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, 
                         #進行l2正規化
                         norm='l2', 
                         smooth_idf=True)
#印出陣列
print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

In [ ]:
#計算TF-IDF的各種值，帶入公式後，算出TF-IDF的值print出來
tf_is = 3
n_docs = 3
idf_is = np.log((n_docs+1) / (3+1))
tfidf_is = tf_is * (idf_is + 1)
print(f'tf-idf of term "is" = {tfidf_is:.2f}')

In [ ]:
#將文檔集合轉換為 TF-IDF 矩陣
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(docs)).toarray()[-1]
raw_tfidf 

In [ ]:
#計算了最後一個文檔的 TF-IDF 值進行 l2 正規化後的結果
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

In [ ]:
#第一個評論的最後 50 個字
df.loc[0, 'review'][-50:]

In [ ]:
#去除特定符號，將他們值轉換成字串
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [ ]:
#對第一行使用preprocessor去除符號
preprocessor(df.loc[0, 'review'][-50:])

In [ ]:
#對輸入的資料都做preprocessor處理
preprocessor("</a>This :) is :( a test :-)!")

In [ ]:
#將保存review列的做preprocessor處理
df['review'] = df['review'].apply(preprocessor)

In [ ]:
#使用PorterStemmer對每個單詞提取其詞幹後返回一個包含詞幹的單詞列表
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [ ]:
#將輸入的文本按空格分割成單詞列表
tokenizer('runners like running and thus they run')

In [ ]:
#對每個單詞使用 Porter stemmer 進行詞幹提取
tokenizer_porter('runners like running and thus they run')

In [ ]:
#下載停用詞
import nltk

nltk.download('stopwords')

In [ ]:
#過濾掉文本中的停用詞
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')
 if w not in stop]

In [ ]:
#分為訓練集和測試集
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [ ]:
#建立Pipeline，跟之前一樣
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
#使用兩個網格，各有不同的參數
"""
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]
"""

small_param_grid = [{'vect__ngram_range': [(1, 1)],
                     'vect__stop_words': [None],
                     'vect__tokenizer': [tokenizer, tokenizer_porter],
                     'clf__penalty': ['l2'],
                     'clf__C': [1.0, 10.0]},
                    {'vect__ngram_range': [(1, 1)],
                     'vect__stop_words': [stop, None],
                     'vect__tokenizer': [tokenizer],
                     'vect__use_idf':[False],
                     'vect__norm':[None],
                     'clf__penalty': ['l2'],
                  'clf__C': [1.0, 10.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [ ]:
#進行訓練
gs_lr_tfidf.fit(X_train, y_train)

In [ ]:
print(f'Best parameter set: {gs_lr_tfidf.best_params_}')
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')

In [ ]:
#最佳模型從網格搜索 gs_lr_tfidf 中取出
clf = gs_lr_tfidf.best_estimator_
print(f'Test Accuracy: {clf.score(X_test, y_test):.3f}')

In [ ]:
#使用了 Stratified K-Fold 交叉驗證，評估 Logistic 迴歸模型的性能
from sklearn.linear_model import LogisticRegression
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

np.random.seed(0)
np.set_printoptions(precision=6)
y = [np.random.randint(3) for i in range(25)]
X = (y + np.random.randn(25)).reshape(-1, 1)

cv5_idx = list(StratifiedKFold(n_splits=5, shuffle=False).split(X, y))
    
lr = LogisticRegression()
cross_val_score(lr, X, y, cv=cv5_idx)

In [ ]:
#使用了 GridSearchCV 來對 Logistic 進行網格搜索
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression()
gs = GridSearchCV(lr, {}, cv=cv5_idx, verbose=3).fit(X, y) 

In [ ]:
#找最好的
gs.best_score_

In [ ]:
#使用了交叉驗證來評估模型的性能
lr = LogisticRegression()
cross_val_score(lr, X, y, cv=cv5_idx).mean()

In [ ]:
#定義函式
import numpy as np
import re
from nltk.corpus import stopwords


# The `stop` is defined as earlier in this chapter
# Added it here for convenience, so that this section
# can be run as standalone without executing prior code
# in the directory
stop = stopwords.words('english')

#保留了表情符號，將文本轉換為小寫，去除了非字母、非數字、非底線的字符，過濾了停用詞
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

#逐行生成文本和對應的標籤
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [ ]:
#讀CSV的數據
next(stream_docs(path='movie_data.csv'))

In [ ]:
#定義get_minibatch，讀取小量的資料
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [ ]:
#使用HashingVectorizer 將文本數據轉換為哈希特徵向量
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier


vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

In [ ]:
#創建了一個 SGDClassifier 分類器，並且讀取movie_data.csv
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

clf = SGDClassifier(loss='log', random_state=1)


doc_stream = stream_docs(path='movie_data.csv')

In [ ]:
#跟上面確認進度差不多，但有使用 HashingVectorizer 將文本轉換為特徵向量 X_train
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

In [ ]:
#讀一個5000 個文字的X_test, y_test，使用 HashingVectorizer轉換特徵向量
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print(f'Accuracy: {clf.score(X_test, y_test):.3f}')

In [ ]:
#跑訓練
clf = clf.partial_fit(X_test, y_test)

In [ ]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')

# the following is necessary on some computers:
df = df.rename(columns={"0": "review", "1": "sentiment"})

df.head(3)

In [ ]:
#使用 CountVectorizer 來將電影評論文本轉換為詞頻矩陣
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['review'].values)

In [ ]:
#使用 Latent Dirichlet Allocation (LDA) 模型對詞頻矩陣 X 進行主題建模
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)

In [ ]:
lda.components_.shape

In [ ]:

n_top_words = 5
feature_names = count.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {(topic_idx + 1)}:')
    print(' '.join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))

In [ ]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print(f'\nHorror movie #{(iter_idx + 1)}:')
    print(df['review'][movie_idx][:300], '...')